In [ ]:
!pip install scikit_posthocs

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from statsmodels.formula.api import ols
from scipy.stats import shapiro
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
import scikit_posthocs as sp

### Carga de datos


In [ ]:
nataniel = "https://raw.githubusercontent.com/MarcoGonzalezS/Project-data/main/Analisis/Datos_104_analisis.csv"

In [ ]:
df_nataniel = pd.read_csv(nataniel, sep = ';',
                         parse_dates=['date'],
                 dtype={'Promocion': 'string',
                        'Promocion_cod':'category',
                        'Tipo_promocion':'string',
                        'Tipo_promocion_cod':'category',
                        'Feriado_cod': 'category',
                        'Tipo_feriado_cod':'category',
                        'Mes': 'string',
                        'Funcionamineto_cod': 'category',
                        'Fin_de_semana_cod':'category',
                        'Año_semana': 'string',
                        'Estacion_cod': 'category'})

### Dataset por hora

In [ ]:
df_nataniel.head()

,date,Promocion,Promocion_cod,Tipo_promocion,Tipo_promocion_cod,Feriado,Feriado_cod,Tipo_feriado,Tipo_feriado_cod,Año,...,Numero_semana,Fin_de_semana,Fin_de_semana_cod,Hora,Funcionamiento,Funcionamiento_cod,Año_semana,Estacion,Estacion_cod,Ordenes
0,2022-01-01 00:00:00,No,0,No aplica,0,Si,1,Año Nuevo,1,2022,...,52,Si,1,0,Cerrado,0,202152,Verano,1,0
1,2022-01-01 01:00:00,No,0,No aplica,0,Si,1,Año Nuevo,1,2022,...,52,Si,1,1,Cerrado,0,202152,Verano,1,0
2,2022-01-01 02:00:00,No,0,No aplica,0,Si,1,Año Nuevo,1,2022,...,52,Si,1,2,Cerrado,0,202152,Verano,1,0
3,2022-01-01 03:00:00,No,0,No aplica,0,Si,1,Año Nuevo,1,2022,...,52,Si,1,3,Cerrado,0,202152,Verano,1,0
4,2022-01-01 04:00:00,No,0,No aplica,0,Si,1,Año Nuevo,1,2022,...,52,Si,1,4,Cerrado,0,202152,Verano,1,0


In [ ]:
px.line(df_nataniel,x=df_nataniel['date'],y=df_nataniel['Ordenes'],title='Pedidos por hora')

In [ ]:
df_semana = df_nataniel.groupby(pd.Grouper(key='date', axis=0,
                      freq='W')).agg({'Año':'first','Numero_semana':'first','Año_semana':'first','Ordenes':'sum'})
px.line(df_semana,x='Año_semana',y='Ordenes',title='Gráfico ordenes por semana según año')

In [ ]:
df_mes = df_nataniel.groupby(pd.Grouper(key='date', axis=0,
                      freq='M')).agg({'Año':'first','Mes':'first','Ordenes':'sum'})
px.line(df_mes,x='Mes',y='Ordenes',color='Año',title='Ordenes por mes según año')

###Creación de dataset por día

In [ ]:
df_nataniel_dia = df_nataniel.query('Funcionamiento_cod == 1')
df_nataniel_dia = df_nataniel_dia.groupby(pd.Grouper(key='date', axis=0,
                      freq='D')).agg({'Promocion':'first','Promocion_cod':'first','Tipo_promocion':'last','Tipo_promocion_cod':'last',
                                      'Feriado':'first', 'Feriado_cod':'first','Tipo_feriado_cod':'first',
                                      'Tipo_feriado':'first','Tipo_feriado_cod':'first','Año':'first','Mes':'first','Mes_cod':'first',
                                      'Dia_mes':'first','Dia_semana':'first','Dia_semana_cod':'first','Numero_semana':'first',
                                      'Fin_de_semana':'first','Fin_de_semana_cod':'first','Funcionamiento_cod':'first','Estacion':'first','Estacion_cod':'first',
                                      'Año_semana':'first','Ordenes':'sum'})

In [ ]:
#Se crea dataset agregrado por semana, eliminando el primer registro ya que corresponde a la semana 52 del año 2021 que no es considerado
df_semana = df_nataniel.groupby(pd.Grouper(key='date', axis=0,
                      freq='W')).agg({'Año':'first','Numero_semana':'first','Año_semana':'first','Ordenes':'sum'})
df_semana.drop(index='2022-01-02',axis=0,inplace=True)
px.line(df_semana,x='Numero_semana',y='Ordenes',color='Año',title='Gráfico ordenes por semana según año')

### Análisis por variable

Para cada variable del dataset, se realiza boxplot para comparar la media de sus grupos y test de hipótesis para verificar si existe diferencia significativa.

En variables categóricas, para comparar las medias de las ordenes se utiliza ANOVA. Para esto, se debe verificar que se cumplan los siguientes supuestos:
- Observaciones deben ser independientes entre si
- Los residuos deben seguir una distribución normal
- Debe cumplirse la homogeneidad de varianzas

A continuación se verifica la normalidad de los residuos de las órdenes por día de las variables categóricas

In [ ]:
model = ols('Ordenes ~ C(Promocion_cod) + C(Tipo_promocion_cod) + C(Feriado_cod) + C(Mes_cod) + C(Dia_semana_cod) + C(Numero_semana) + C(Funcionamiento_cod) + C(Estacion_cod) + C(Fin_de_semana_cod)', data=df_nataniel_dia).fit()
residuals = model.resid
shapiro_test = shapiro(residuals)
print(f'Shapiro-Wilk test: W={shapiro_test[0]}, p-value={shapiro_test[1]}')

Shapiro-Wilk test: W=0.9933105707168579, p-value=0.0027436180971562862


Como el p-valor es menor que 0,05 se rechaza la hipótesis nula, por lo que los residuos no están normalmente distribuidos.
No se puede realizar ANOVA, por lo que se implementará su verisón no paramétrica Mann Whitney y su extensión Kruskal Wallis

#### **Promoción**

In [ ]:
fig = px.box(df_nataniel_dia, x="Promocion", y="Ordenes",title='Ordenes según existencia de promoción')
fig.show()

Test de Mann Whitney para verificar si hay diferencia significativa con la presencia de promociones

In [ ]:
ordenes = df_nataniel_dia.groupby('Promocion_cod')['Ordenes'].apply(list).tolist()
ordenes_sin_promo, ordenes_promo = ordenes

In [ ]:
mann_whitney_test = mannwhitneyu(ordenes_sin_promo, ordenes_promo, alternative='two-sided')
print(f'Mann-Whitney U test: U={mann_whitney_test.statistic}, p-value={mann_whitney_test.pvalue}')

Mann-Whitney U test: U=38327.0, p-value=2.59843778634049e-07


Si hay diferencia significativa en las distribuciones

#### **Tipo de promoción**

In [ ]:
fig = px.box(df_nataniel_dia, x="Tipo_promocion", y="Ordenes",title='Ordenes según tipo de promoción')
fig.show()

Test de Kruskal Wallis para comparar tipos de promociones

In [ ]:
ordenes_tipo_promo = [group['Ordenes'].tolist() for name, group in df_nataniel_dia.groupby('Tipo_promocion_cod')]
kruskal_test = kruskal(*ordenes_tipo_promo)
print(f'Kruskal-Wallis test: H={kruskal_test[0]}, p-value={kruskal_test[1]}')

Kruskal-Wallis test: H=78.27712010429217, p-value=3.174824326003121e-12


Hay diferencia en al menos uno de los grupos, se debe evaluar cuales tienen diferencia a través de post hoc

In [ ]:
df_promocion = [df_nataniel_dia[df_nataniel_dia['Tipo_promocion_cod'] == "0"]['Ordenes'],
                df_nataniel_dia[df_nataniel_dia['Tipo_promocion_cod'] == "1"]['Ordenes'],
                df_nataniel_dia[df_nataniel_dia['Tipo_promocion_cod'] == "2"]['Ordenes'],
                df_nataniel_dia[df_nataniel_dia['Tipo_promocion_cod'] == "3"]['Ordenes'],
                df_nataniel_dia[df_nataniel_dia['Tipo_promocion_cod'] == "4"]['Ordenes'],
                df_nataniel_dia[df_nataniel_dia['Tipo_promocion_cod'] == "5"]['Ordenes'],
                df_nataniel_dia[df_nataniel_dia['Tipo_promocion_cod'] == "6"]['Ordenes'],
                df_nataniel_dia[df_nataniel_dia['Tipo_promocion_cod'] == "7"]['Ordenes'],
                df_nataniel_dia[df_nataniel_dia['Tipo_promocion_cod'] == "8"]['Ordenes'],
                df_nataniel_dia[df_nataniel_dia['Tipo_promocion_cod'] == "9"]['Ordenes'],
                df_nataniel_dia[df_nataniel_dia['Tipo_promocion_cod'] == "10"]['Ordenes'],
                df_nataniel_dia[df_nataniel_dia['Tipo_promocion_cod'] == "11"]['Ordenes']]
dunn_test = sp.posthoc_dunn(df_promocion)

In [ ]:
group_names= ["Sin promoción", "Black","Blackout","Cyber","Días T","T week","Post black","Post Cyber","Promo año nuevo","Promo fiestas patrias","Promo navidad","Venta nocturna"]
dunn_test.columns= group_names
dunn_test.index= group_names
dunn_test.style.applymap(lambda x: "background-color:red" if x<0.05 else "background-color: white")

,Sin promoción,Black,Blackout,Cyber,Días T,T week,Post black,Post Cyber,Promo año nuevo,Promo fiestas patrias,Promo navidad,Venta nocturna
Sin promoción,1.000000,0.017520,0.098939,0.000000,0.550813,0.820188,0.560456,0.181336,0.021441,0.000000,0.013015,0.040784
Black,0.017520,1.000000,0.588804,0.000715,0.045241,0.029052,0.113622,0.997480,0.534579,0.076946,0.477890,0.586553
Blackout,0.098939,0.588804,1.000000,0.084948,0.080721,0.096575,0.105383,0.641730,0.939995,0.672905,0.975578,0.923377
Cyber,0.000000,0.000715,0.084948,1.000000,0.000002,0.000000,0.000148,0.009595,0.021791,0.034932,0.022254,0.027727
Días T,0.550813,0.045241,0.080721,0.000002,1.000000,0.668267,0.846383,0.146018,0.028429,0.000276,0.020700,0.042837
T week,0.820188,0.029052,0.096575,0.000000,0.668267,1.000000,0.624610,0.176921,0.025114,0.000007,0.016348,0.043721
Post black,0.560456,0.113622,0.105383,0.000148,0.846383,0.624610,1.000000,0.185355,0.064024,0.007805,0.053796,0.078638
Post Cyber,0.181336,0.997480,0.641730,0.009595,0.146018,0.176921,0.185355,1.000000,0.628761,0.236167,0.585455,0.663216
Promo año nuevo,0.021441,0.534579,0.939995,0.021791,0.028429,0.025114,0.064024,0.628761,1.000000,0.485798,0.953567,0.976844
Promo fiestas patrias,0.000000,0.076946,0.672905,0.034932,0.000276,0.000007,0.007805,0.236167,0.485798,1.000000,0.517065,0.496354


Se observa que las promociones que presentan distinta distribución en las ordenes son las siguientes:
- Black
- Cyber
- Promo año nuevo
- Promo fiestas patrias
- Promo navidad
- Venta nocturna

#### **Feriado**

In [ ]:
fig = px.box(df_nataniel_dia, x="Feriado", y="Ordenes",title='Ordenes según existencia de feriado')
fig.show()

Test de Mann Whitney para verificar si hay diferencia en la distribución de las ordenes con la presencia de feriados

In [ ]:
ordenes = df_nataniel_dia.groupby('Feriado_cod')['Ordenes'].apply(list).tolist()
ordenes_no_feriado, ordenes_feriado = ordenes

In [ ]:
mann_whitney_test = mannwhitneyu(ordenes_no_feriado, ordenes_feriado, alternative='two-sided')
print(f'Mann-Whitney U test: U={mann_whitney_test.statistic}, p-value={mann_whitney_test.pvalue}')

Mann-Whitney U test: U=7998.0, p-value=0.5219242963733007


Como el p-valor es mayor a 0,05, no se rechaza la hipótesis nula. No hay diferencia significativas en las ordenes según los feriados

#### **Tipo de feriado**

In [ ]:
fig = px.box(df_nataniel_dia, x="Tipo_feriado", y="Ordenes",title='Ordenes según tipo de feriado')
fig.show()

Test de Kruskal Wallis para evaluar si hay diferencia significativa en las ordenes en días feriados

In [ ]:
ordenes_tipo_feriado = [group['Ordenes'].tolist() for name, group in df_nataniel_dia.groupby('Tipo_feriado')]
kruskal_test = kruskal(*ordenes_tipo_feriado)
print(f'Kruskal-Wallis test: H={kruskal_test[0]}, p-value={kruskal_test[1]}')

Kruskal-Wallis test: H=11.154588245050874, p-value=0.5978689822203294


In [ ]:
ordenes_dia_mes = [group['Ordenes'].tolist() for name, group in df_nataniel_dia.groupby('Dia_mes')]
kruskal_test = kruskal(*ordenes_dia_mes)
print(f'Kruskal-Wallis test: H={kruskal_test[0]}, p-value={kruskal_test[1]}')

Kruskal-Wallis test: H=68.34543516108167, p-value=8.057358859074949e-05


Si bien el p valor indica que hay diferencia significativa en la distribución en los grupos, no se considera esta variable ya que no era significativo si existe feriado o no.

#### **Estación**

In [ ]:
fig = px.box(df_nataniel_dia, x="Estacion", y="Ordenes",title='Ordenes según estación del año')
fig.show()

In [ ]:
px.histogram(df_nataniel_dia,x=df_nataniel_dia['Estacion'],y=df_nataniel_dia['Ordenes'])

Test de Kruskal Wallis para comparar la distribución de las ordenes entre los grupos

In [ ]:
ordenes_por_estacion = [group['Ordenes'].tolist() for name, group in df_nataniel_dia.groupby('Estacion_cod')]
kruskal_test = kruskal(*ordenes_por_estacion)
print(f'Kruskal-Wallis test: H={kruskal_test[0]}, p-value={kruskal_test[1]}')

Kruskal-Wallis test: H=62.17837643365295, p-value=2.012427875598301e-13


Como el p-valor es menor a 0,05, se infiere que hay diferencia significativa en las distribución de las ordenes según estación




#### **Año**

In [ ]:
fig = px.box(df_nataniel_dia, x="Año", y="Ordenes",title='Ordenes por año')
fig.show()

In [ ]:
px.histogram(df_nataniel_dia,x=df_nataniel_dia['Año'],y=df_nataniel_dia['Ordenes'])

Se observa una disminución en la demanda durante el 2023

#### **Mes**

In [ ]:
fig = px.box(df_nataniel_dia, x="Mes", y="Ordenes",title='Ordenes por mes')
fig.show()

In [ ]:
px.histogram(df_nataniel_dia,x=df_nataniel_dia['Mes'],y=df_nataniel_dia['Ordenes'])

In [ ]:
fig = px.box(df_nataniel_dia, x="Mes", y="Ordenes",title='Ordenes por mes',color='Año')
fig.show()

Test de Kruskal Wallis para ver si hay diferencia en la distribución de ordenes por mes

In [ ]:
ordenes_por_mes = [group['Ordenes'].tolist() for name, group in df_nataniel_dia.groupby('Mes_cod')]
kruskal_test = kruskal(*ordenes_por_mes)
print(f'Kruskal-Wallis test: H={kruskal_test[0]}, p-value={kruskal_test[1]}')

Kruskal-Wallis test: H=101.12288953453529, p-value=1.0699832419164073e-16


Como el p-valor es menor a 0,05, se infiere que hay diferencia significativa entre los grupos


#### **Día del mes**

In [ ]:
fig = px.box(df_nataniel_dia, x="Dia_mes", y="Ordenes",title='Ordenes por día del mes')
fig.show()

In [ ]:
ordenes_por_dia_mes = [group['Ordenes'].tolist() for name, group in df_nataniel_dia.groupby('Dia_mes')]
kruskal_test = kruskal(*ordenes_por_dia_mes)
print(f'Kruskal-Wallis test: H={kruskal_test[0]}, p-value={kruskal_test[1]}')

Kruskal-Wallis test: H=68.34543516108167, p-value=8.057358859074949e-05


#### **Día de la semana**

In [ ]:
fig = px.box(df_nataniel_dia, x="Dia_semana", y="Ordenes",title='Ordenes por día de la semana',
             category_orders={"Dia_semana":["Lunes","Martes","Miércoles","Jueves","Viernes","Sábado","Domingo"]})
fig.show()

In [ ]:
px.histogram(df_nataniel_dia,x=df_nataniel_dia['Dia_semana'],y=df_nataniel_dia['Ordenes'],
             category_orders={"Dia_semana":["Lunes","Martes","Miércoles","Jueves","Viernes","Sábado","Domingo"]})

Test de Kruskal Wallis para comparar si existe diferencia significativa en la distribución de las ordenes según el día de la semana

In [ ]:
ordenes_por_dia = [group['Ordenes'].tolist() for name, group in df_nataniel_dia.groupby('Dia_semana_cod')]
kruskal_test = kruskal(*ordenes_por_dia)
print(f'Kruskal-Wallis test: H={kruskal_test[0]}, p-value={kruskal_test[1]}')

Kruskal-Wallis test: H=147.08150175330968, p-value=3.202531316430447e-29


Como el p-valor es menor a 0,05, se infiere que hay diferencia significativa entre los grupos

#### **Número de semana**

In [ ]:
fig = px.box(df_nataniel_dia, x="Numero_semana", y="Ordenes",title='Ordenes por número de semana')
fig.show()

In [ ]:
ordenes_por_num = [group['Ordenes'].tolist() for name, group in df_nataniel_dia.groupby('Numero_semana')]
kruskal_test = kruskal(*ordenes_por_num)
print(f'Kruskal-Wallis test: H={kruskal_test[0]}, p-value={kruskal_test[1]}')

Kruskal-Wallis test: H=176.13349860502544, p-value=1.1234360549845648e-15


El p valor es menor a 0,05, hay diferencia significativa entre las distribuciones en los grupos

#### **Fin de semana**

In [ ]:
fig = px.box(df_nataniel_dia, x="Fin_de_semana", y="Ordenes",title='Ordenes fin de semana versus dia de semana')
fig.show()

Test de Mann Whitney para comparar las distribuciones en las ordenes durante días de semana o feriado.

In [ ]:
ordenes = df_nataniel_dia.groupby('Fin_de_semana')['Ordenes'].apply(list).tolist()
ordenes_fin_de_semana, ordenes_semana = ordenes

In [ ]:
mann_whitney_test = mannwhitneyu(ordenes_fin_de_semana, ordenes_semana, alternative='two-sided')
print(f'Mann-Whitney U test: U={mann_whitney_test.statistic}, p-value={mann_whitney_test.pvalue}')

Mann-Whitney U test: U=33926.5, p-value=4.15695458628834e-13


p-valor es menor a 0,05, se rechaza la hipotesis nula y si hay diferencia significativa entre ambos grupos.

#### **Ordenes por hora**

In [ ]:
fig = px.box(df_nataniel.query('Funcionamiento_cod == 1'), x="Hora", y="Ordenes",title='Ordenes por hora')
fig.show()

## Análisis promociones
Considerando Black, Cyber, Año nuevo, Fiestas patrias, Navidad y Venta nocturna.

In [ ]:
df_nuevo = pd.read_csv("https://raw.githubusercontent.com/MarcoGonzalezS/Project-data/main/Modelo_1/Analisis/Datos_104_1.csv", sep = ';',
                         parse_dates=['date'],
                 dtype={'Promocion': 'string',
                        'Promocion_cod':'category',
                        'Tipo_promocion':'string',
                        'Tipo_promocion_cod':'category',
                        'Mes': 'string',
                        'Funcionamineto_cod': 'category',
                        'Fin_de_semana_cod':'category',
                        'Año_semana': 'string',
                        'Estacion_cod': 'category'})

In [ ]:
df_nuevo_dia = df_nuevo.query('Funcionamiento_cod == 1')
df_nuevo_dia = df_nuevo_dia.groupby(pd.Grouper(key='date', axis=0,
                      freq='D')).agg({'Promocion_cod':'last','Promocion':'last','Tipo_promocion':'last','Tipo_promocion_cod':'last',
                                      'Año':'first','Mes_cod':'first',
                                      'Dia_mes':'first','Dia_semana_cod':'first','Numero_semana':'first',
                                      'Fin_de_semana_cod':'first','Funcionamiento_cod':'first','Estacion_cod':'first',
                                      'Año_semana':'first','Ordenes':'sum'})

In [ ]:
fig = px.box(df_nuevo_dia.query('Funcionamiento_cod == 1'), x="Promocion", y="Ordenes",title='Ordenes según existencia de promoción')
fig.show()

In [ ]:
fig = px.box(df_nuevo_dia.query('Funcionamiento_cod == 1'), x="Tipo_promocion", y="Ordenes",title='Ordenes según tipo de promoción')
fig.show()

In [ ]:
ordenes_nuevas = [group['Ordenes'].tolist() for name, group in df_nuevo_dia.groupby('Tipo_promocion_cod')]
kruskal_test = kruskal(*ordenes_nuevas)
print(f'Kruskal-Wallis test: H={kruskal_test[0]}, p-value={kruskal_test[1]}')

Kruskal-Wallis test: H=72.9496176765076, p-value=1.0137941635525247e-13


In [ ]:
df = [df_nuevo_dia[df_nuevo_dia['Tipo_promocion_cod'] == "0"]['Ordenes'],
                df_nuevo_dia[df_nuevo_dia['Tipo_promocion_cod'] == "1"]['Ordenes'],
                df_nuevo_dia[df_nuevo_dia['Tipo_promocion_cod'] == "2"]['Ordenes'],
                df_nuevo_dia[df_nuevo_dia['Tipo_promocion_cod'] == "3"]['Ordenes'],
                df_nuevo_dia[df_nuevo_dia['Tipo_promocion_cod'] == "4"]['Ordenes'],
                df_nuevo_dia[df_nuevo_dia['Tipo_promocion_cod'] == "5"]['Ordenes'],
                df_nuevo_dia[df_nuevo_dia['Tipo_promocion_cod'] == "6"]['Ordenes']]

In [ ]:
dunn_test = sp.posthoc_dunn(df)
group_names= ["Sin promoción", "Black","Cyber","Promo año nuevo","Promo fiestas patrias","Promo navidad", "Venta nocturna"]
dunn_test.columns= group_names
dunn_test.index= group_names
dunn_test.style.applymap(lambda x: "background-color:red" if x<0.05 else "background-color: white")

,Sin promoción,Black,Cyber,Promo año nuevo,Promo fiestas patrias,Promo navidad,Venta nocturna
Sin promoción,1.000000,0.017716,0.000000,0.021721,0.000000,0.041294,0.013185
Black,0.017716,1.000000,0.000715,0.534579,0.076946,0.586553,0.477890
Cyber,0.000000,0.000715,1.000000,0.021791,0.034932,0.027727,0.022254
Promo año nuevo,0.021721,0.534579,0.021791,1.000000,0.485798,0.976844,0.953567
Promo fiestas patrias,0.000000,0.076946,0.034932,0.485798,1.000000,0.496354,0.517065
Promo navidad,0.041294,0.586553,0.027727,0.976844,0.496354,1.000000,0.932495
Venta nocturna,0.013185,0.477890,0.022254,0.953567,0.517065,0.932495,1.000000
